In [1]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0, verbose=True)

In [ ]:
llm.invoke("Hi! What is your name?")

# '\n\nI am an AI digital assistant and do not have a name. You can call me OpenAI. What can I assist you with?'

In [13]:
# Install sqlite3
# Create the folder notebooks

from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///notebooks/Chinook.db")

# cd notebooks
# sqlite3 Chinook.db
# SELECT name FROM sqlite_master WHERE type='table';

In [14]:
# Chinook.db comes from here: https://github.com/lerocha/chinook-database
# 
# Step 1: Create the database.
# sqlite3 Chinook.db
# 
# Step 2: Create the tables for the database.
#
# Download the database definition and content for sqlite from 
# - https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql
# 
# Run:
# sqlite3 -init Chinook_Sqlite.sql Chinook.db
# 
# Done: .quit

In [ ]:
print(db.get_table_info())
# CREATE TABLE "Album" (
# 	"AlbumId" INTEGER NOT NULL, 
# 	"Title" NVARCHAR(160) NOT NULL, 
# 	"ArtistId" INTEGER NOT NULL, 
# 	PRIMARY KEY ("AlbumId"), 
# 	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
# )
# ...

In [16]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
ai_msg = db_chain.invoke("How many tables is there in the database?")

# > Entering new SQLDatabaseChain chain...
# How many tables is there in the database?
# SQLQuery:SELECT COUNT(*) FROM sqlite_master WHERE type='table'
# SQLResult: [(0,)]
# Answer:0
# > Finished chain.

# > Entering new SQLDatabaseChain chain...
# How many tables is there in the database?
# SQLQuery:SELECT COUNT(*) FROM sqlite_master WHERE type='table'
# SQLResult: [(11,)]
# Answer:There are 11 tables in the database.
# > Finished chain.

In [ ]:
ai_msg
# {'query': 'How many tables is there in the database?', 'result': '0'}
# {'query': 'How many tables is there in the database?', 'result': 'There are 11 tables in the database.'}

In [ ]:
ai_msg['result']
# '0'
# 'There are 11 tables in the database.'

In [ ]:
ai_msg = db_chain.invoke("How many employees are there?")

# > Entering new SQLDatabaseChain chain...
# How many employees are there?
# SQLQuery:SELECT COUNT(*) FROM "employees"
# ...
# OperationalError: (sqlite3.OperationalError) no such table: employees
# [SQL: SELECT COUNT(*) FROM "employees"]

# > Entering new SQLDatabaseChain chain...
# How many employees are there?
# SQLQuery:SELECT COUNT(*) FROM Employee
# SQLResult: [(8,)]
# Answer:There are 8 employees.
# > Finished chain.

In [21]:
# Use Query Checker
# Sometimes the Language Model generates invalid SQL with small mistakes that can be self-corrected using the same technique used by
# the SQL Database Agent to try and fix the SQL using the LLM. You can simply specify this option when creating the DB chain.

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)

In [ ]:
ai_msg = db_chain.invoke("How many albums by Aerosmith?")
# > Entering new SQLDatabaseChain chain...
# How many albums by Aerosmith?
# SQLQuery:SELECT COUNT(*) FROM albums WHERE artist_id = (SELECT id FROM artists WHERE name = "Aerosmith")
# ...
# OperationalError: (sqlite3.OperationalError) no such table: albums
# [SQL: SELECT COUNT(*) FROM albums WHERE artist_id = (SELECT id FROM artists WHERE name = "Aerosmith")]

# > Entering new SQLDatabaseChain chain...
# How many albums by Aerosmith?
# SQLQuery:SELECT COUNT(*) FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = "Aerosmith")
# SQLResult: [(1,)]
# Answer:1
# > Finished chain.

In [ ]:
print(ai_msg)
# {'query': 'How many albums by Aerosmith?', 'result': '1'}

print(ai_msg['result'])
# 1

In [ ]:
ai_msg = db_chain.invoke("List the name of all the tables in the database")

# > Entering new SQLDatabaseChain chain...
# List the name of all the tables in the database
# SQLQuery:SELECT name FROM sqlite_master WHERE type='table'
# SQLResult: [('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
# Answer:Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
# > Finished chain.

In [ ]:
ai_msg = db_chain.invoke("List the column names of MediaType table")

# > Entering new SQLDatabaseChain chain...
# List the column names of MediaType table
# SQLQuery:SELECT "MediaTypeId", "Name" FROM "MediaType" LIMIT 5
# SQLResult: [(1, 'MPEG audio file'), (2, 'Protected AAC audio file'), (3, 'Protected MPEG-4 video file'), (4, 'Purchased AAC audio file'), (5, 'AAC audio file')]
# Answer:MediaTypeId, Name
# > Finished chain.

In [ ]:
ai_msg = db_chain.invoke("How many unique names are there in Name column of the MediaType table?")

# > Entering new SQLDatabaseChain chain...
# How many unique names are there in Name column of the MediaType table?
# SQLQuery:SELECT COUNT(DISTINCT "Name") FROM "MediaType"
# SQLResult: [(5,)]
# Answer:There are 5 unique names in the Name column of the MediaType table.
# > Finished chain.


In [ ]:
ai_msg = db_chain.invoke("List all the unique media types of the MediaType table")

# > Entering new SQLDatabaseChain chain...
# List all the unique media types of the MediaType table
# SQLQuery:SELECT DISTINCT "Name" FROM "MediaType"
# SQLResult: [('MPEG audio file',), ('Protected AAC audio file',), ('Protected MPEG-4 video file',), ('Purchased AAC audio file',), ('AAC audio file',)]
# Answer:MPEG audio file, Protected AAC audio file, Protected MPEG-4 video file, Purchased AAC audio file, AAC audio file
# > Finished chain.

In [ ]:
ai_msg = db_chain.invoke("List the structure of the table employees")

# > Entering new SQLDatabaseChain chain...
# List the structure of the table employees
# SQLQuery:SELECT * FROM Employee LIMIT 5
# SQLResult: [(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'), (2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'), (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com'), (4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com'), (5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'AB', 'Canada', 'T3B 1Y7', '1 (780) 836-9987', '1 (780) 836-9543', 'steve@chinookcorp.com')]
# Answer:The table employees has the following structure: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email.
# > Finished chain.

In [ ]:
ai_msg = db_chain.invoke("List the structure of the table artist")

# > Entering new SQLDatabaseChain chain...
# List the structure of the table artist
# SQLQuery:SELECT * FROM Artist LIMIT 5
# SQLResult: [(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]
# Answer:The table "Artist" has two columns: "ArtistId" and "Name". The "ArtistId" column is an integer and is the primary key for the table. The "Name" column is a string and contains the name of the artist.
# > Finished chain.


In [ ]:
ai_msg = db_chain.invoke("insert a new artist named 'Raul Seixas'")

# > Entering new SQLDatabaseChain chain...
# insert a new artist named 'Raul Seixas'
# SQLQuery:INSERT INTO "Artist" ("Name") VALUES ('Raul Seixas')
# SQLResult: 
# Answer:No results, but the new artist 'Raul Seixas' has been successfully inserted into the Artist table.
# > Finished chain.

In [ ]:
ai_msg = db_chain.invoke("for the artist 'Raul Seixas', insert the following albums: 'Raul Seixas', 'Novo Aeon', 'Gita'")

# > Entering new SQLDatabaseChain chain...
# for the artist 'Raul Seixas', insert the following albums: 'Raul Seixas', 'Novo Aeon', 'Gita'
# SQLQuery:INSERT INTO "Album" ("Title", "ArtistId") VALUES ('Raul Seixas', (SELECT "ArtistId" FROM "Artist" WHERE "Name" = 'Raul Seixas')), ('Novo Aeon', (SELECT "ArtistId" FROM "Artist" WHERE "Name" = 'Raul Seixas')), ('Gita', (SELECT "ArtistId" FROM "Artist" WHERE "Name" = 'Raul Seixas'))
# SQLResult: 
# Answer:No results, but the albums 'Raul Seixas', 'Novo Aeon', and 'Gita' have been successfully inserted into the Album table for the artist 'Raul Seixas'.
# > Finished chain.

In [ ]:
ai_msg = db_chain.invoke("add a new column 'year' with default value '1970' to the table 'album'")

# > Entering new SQLDatabaseChain chain...
# add a new column 'year' with default value '1970' to the table 'album'
# SQLQuery:ALTER TABLE "Album" ADD COLUMN "year" INTEGER DEFAULT 1970
# SQLResult: 
# Answer:The new column 'year' has been successfully added to the 'Album' table with a default value of '1970'.
# > Finished chain.